In [2]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [7]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [8]:
funded_mnemonics= ["rookie similar wire owner wash once fish mosquito glad coffee family venture adult funny melt gas inspire tuna buzz sell dignity pottery gold able bracket",
                  "narrow mimic suffer top suspect follow menu broccoli try snake meat erase napkin lucky client forget sense bread glad eight uniform bacon code about crack", 
                  "city truth device clog grocery sea safe slide glove borrow swap capable trash shaft vast start space distance calm wire hub crush dose ability army", 
                  "unknown arctic antenna country credit december ill practice lawsuit today athlete rescue exit swarm fitness strong minimum soldier wide coffee vacuum piece coil absorb unable", 
                  "gravity adult destroy demand margin coast culture base adjust east banana certain happy daughter bless fiscal fiscal eye forget inspire brain banner evil able bonus"]

funded_accounts = [{'sk': mnemonic.to_private_key(memo), 'pk': mnemonic.to_public_key(memo)} for memo in funded_mnemonics]

In [9]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

## options in contract storage

In [12]:
from pyteal import *    
    
def approval_program():     
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        Return(Int(1))
    ])    
    
    on_closeout = Return(Int(1))
    on_optin = Return(Int(1))
    
    myStatus = App.globalGetEx(Int(0), Bytes("status"))
    Seq([
        myStatus,
        If(myStatus.hasValue(), myStatus.value(), Bytes("none"))
    ])
    
    
    option_name = Concat(Bytes("option_"), Txn.application_args[1])
    
    option_tally = App.globalGetEx(Int(0), option_name)
    on_vote = Seq([
        option_tally, 
        If(option_tally.hasValue(),
           Seq([App.globalPut(option_name, option_tally.value()+Int(1)), Return(Int(1))]),
           Return(Int(0))
          )
    ])
    
    on_add_option = Seq([
        App.globalPut(option_name, Int(0)),
        Return(Int(1))
    ])
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_opiton"), on_add_option]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [10]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete

### Deploy 

In [13]:
approval_bytes, clear_state_bytes = compile()

In [48]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

decision_name = 'to_be_or_not_to_be2'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=61, num_byte_slices=3)     # maximum sum is 64

on_complete = OnComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[2]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    app_args = [decision_name.encode('utf-8')]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[2]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'CF3TQWKQF6NZXC5UKTYV7EIUL2MITRC5BNWNCFGCWY2GNWNGJ4SA'

In [18]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14217895

### Optin

In [19]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

# funded_accounts[1] has some tokens 
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[1]['pk'], params, app_id)
optin_txid = algod_client.send_transaction(optin_tx.sign(funded_accounts[1]['sk']))

In [20]:
optin_tx_response = algod_client.pending_transaction_info(optin_txid)
optin_tx_response

{'pool-error': '',
 'txn': {'sig': 'vQUZQvSbbDVygEam5LvrrCh2P4vNwBw5UlMhwAj9WxqiVqHnpk30G/zYvj9N8lFf8V0ZnhKdRJ/oKqJ9a1tCAg==',
  'txn': {'apan': 1,
   'apid': 14217895,
   'fee': 217000,
   'fv': 12626784,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 12627784,
   'snd': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'type': 'appl'}}}

### Adding options to the election 

In [21]:
app_args = ["add_opiton".encode("utf-8"), "Zaphod".encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

In [22]:
call_tx_response = algod_client.pending_transaction_info(call_txid)
call_tx_response 

{'pool-error': '',
 'txn': {'sig': 'oAAHBF/osjlWe/5FBsRRzXSCBjyaEnvo/SkXwuQSbQHmxCSnfuAiM4otWkgRvwdSuaoNuck6kq/3KtVR05/0BQ==',
  'txn': {'apaa': ['YWRkX29waXRvbg==', 'WmFwaG9k'],
   'apid': 14217895,
   'fee': 237000,
   'fv': 12626784,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 12627784,
   'snd': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'type': 'appl'}}}

In [38]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [40]:
import base64 
def decode_keys(s):
    for entry in s[0]:
        entry['key'] = base64.b64decode(entry['key']).decode('utf-8')
    return s

In [34]:
global_state = read_global_state(algod_client, funded_accounts[2]['pk'], app_id)
decode_keys(global_state)

In [35]:
global_state

[[{'key': 'Creator',
   'value': {'bytes': '/YwCdgGpi2dAH3N8r0McGQR3y/ICpC8JgRo0IzwoxF8=',
    'type': 1,
    'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'dG9fYmVfb3Jfbm90X3RvX2JlMg==', 'type': 1, 'uint': 0}},
  {'key': 'option_Zaphod', 'value': {'bytes': '', 'type': 2, 'uint': 0}}]]

### How many can we add? - let's break this

In [38]:
def add_option_tx(option_name, sender=1):
    app_args = ["add_opiton".encode("utf-8"), option_name.encode("utf-8")]

    # create unsigned transaction
    call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
    call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))
    return call_txid

In [40]:
call_txid = add_option_tx("option2")

In [42]:
global_state = decode_keys(read_global_state(algod_client, funded_accounts[2]['pk'], app_id))
global_state

[[{'key': 'option_option1', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option2', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_Zaphod', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'dG9fYmVfb3Jfbm90X3RvX2JlMg==', 'type': 1, 'uint': 0}},
  {'key': 'Creator',
   'value': {'bytes': '/YwCdgGpi2dAH3N8r0McGQR3y/ICpC8JgRo0IzwoxF8=',
    'type': 1,
    'uint': 0}}]]

In [43]:
call_txid = add_option_tx("option3")
global_state = decode_keys(read_global_state(algod_client, funded_accounts[2]['pk'], app_id))
global_state

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction BSJC5JQK7EFVCY2C6TYJG2GBYGBIV2WJOI7GWCCTNTOKMAA3EXYA: store integer count 4 exceeds schema integer count 3"}


## Entering some options on_create

In [21]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [18]:
from pyteal import *    
    
def approval_program():     
    arg_num = Txn.application_args.length()
    
    on_closeout = Return(Int(1))
    on_optin = Return(Int(1))
    
    on_vote = Seq([ 
        If(App.globalGetEx(Int(0), Concat(Bytes("option_"), Txn.application_args[1])).hasValue(),
           Seq([
               App.globalPut(
                   Concat(Bytes("option_"), Txn.application_args[1]), 
                   App.globalGetEx(Int(0), Concat(Bytes("option_"), Txn.application_args[1])).value() + Int(1)),
               Return(Int(1))
           ]),
           Return(Int(0))
          )
    ])
    
    on_add_options =  Seq([
        If(Txn.application_args[1] != Bytes("NULL_OPTION"), 
           Seq([App.globalPut(Concat(Bytes("option_"), Txn.application_args[1]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[2] != Bytes("NULL_OPTION"), 
          Seq([App.globalPut(Concat(Bytes("option_"), Txn.application_args[2]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[3] != Bytes("NULL_OPTION"), 
          Seq([App.globalPut(Concat(Bytes("option_"), Txn.application_args[3]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[4] != Bytes("NULL_OPTION"), 
          Seq([App.globalPut(Concat(Bytes("option_"), Txn.application_args[4]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[5] != Bytes("NULL_OPTION"), 
          Seq([App.globalPut(Concat(Bytes("option_"), Txn.application_args[5]), Int(0))]),
           Return(Int(1))
        ), 
        Return(Int(1))
    ])
    
    
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        on_add_options,   # record the options 
        Return(Int(1))
    ])    
    
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_options"), on_add_options]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [19]:
approval_bytes, clear_state_bytes = compile()

In [20]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

decision_name = 'thisshouldwork'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=61, num_byte_slices=3)     # maximum sum is 64

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[3]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    # you always need to submit this many options. you can use NULL_OPTION to ingore an option 
    app_args = [decision_name.encode('utf-8'), 
                "option1".encode('utf-8'),
                "option2".encode('utf-8'),
                "option3".encode('utf-8'), 
                "NULL_OPTION".encode('utf-8'),
                "NULL_OPTION".encode('utf-8')]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[3]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'BACQBOAXYPTBNTBV5RKOYNFDLXOKUCTI4EARP4GEYSX7R4YX43GA'

In [139]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14405270

In [140]:
global_state = read_global_state(algod_client, funded_accounts[3]['pk'], app_id)
decode_keys(global_state)

[[{'key': 'Creator',
   'value': {'bytes': 'Vq/fa54I3jnuOB1z755Nmq2wMHqKSdeEauR9B4Vi3QM=',
    'type': 1,
    'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'dGhpc3Nob3VsZHdvcms=', 'type': 1, 'uint': 0}},
  {'key': 'option_option1', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option2', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option3', 'value': {'bytes': '', 'type': 2, 'uint': 0}}]]

### Adding options 5 at a time 

In [143]:
def add_options_tx(option_names, sender=3):
    app_args = ["add_options".encode("utf-8")] + [option_name.encode('utf-8') for option_name in option_names]

    # create unsigned transaction
    call_tx = transaction.ApplicationNoOpTxn(funded_accounts[sender]['pk'], params, app_id, app_args)
    call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[sender]['sk']))
    return call_txid

In [145]:
call_txid = add_options_tx(["new_opt1", "new_opt2", "new_opt3", "new_opt4", "NULL_OPTION"])

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction already in ledger: EUBHHWRQVOFPRCHICBDSHZCTNCNNPPNNIKY2MVIXVKLC3IIQWOXQ"}


In [146]:
global_state = decode_keys(read_global_state(algod_client, funded_accounts[3]['pk'], app_id))
global_state

[[{'key': 'option_option1', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_opt4', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Creator',
   'value': {'bytes': 'Vq/fa54I3jnuOB1z755Nmq2wMHqKSdeEauR9B4Vi3QM=',
    'type': 1,
    'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'dGhpc3Nob3VsZHdvcms=', 'type': 1, 'uint': 0}},
  {'key': 'option_new_opt1', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_opt2', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_opt3', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option2', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option3', 'value': {'bytes': '', 'type': 2, 'uint': 0}}]]

## Voting 

In [95]:
from pyteal import *    
    
def approval_program():     
    VOTING_CREDIT_SYM = Bytes("QVoteDecisionCredits")
    OPTION_PREFIX = Bytes("option_")
    NULL_OPTION = Bytes("NULL_OPTION")
    
    arg_num = Txn.application_args.length()
    
    on_closeout = Return(Int(1))
    
    asset_id = App.globalGet(Bytes("asset_id"))
    asset_balance = AssetHolding.balance(Int(0), asset_id) 
    asset_coeff = App.gobalGet(Bytes("asset_coefficient"))
    on_optin = Seq([     # TODO if not before voting start time 
        asset_balance, 
        If(asset_balance.hasValue(),  # if sender has some tokens, store it, otherwise return 0     
           Seq([App.localPut(Int(0), VOTING_CREDIT_SYM, Mul(asset_balance.value()), asset_coeff), Return(Int(1))]),      
           Return(Int(0))
        ) 
    ])  
    
    option0 = Concat(OPTION_PREFIX, Txn.application_args[1])
    option0_tally = App.globalGetEx(Int(0), option0)            # current total of votes for this option
    option0_votes = Btoi(Txn.application_args[2])               # how much the user wants to vote for this option 
    credit_balance = App.localGet(Int(0), VOTING_CREDIT_SYM)
    on_vote = Seq([ 
        option0_tally, 
        If(option0_tally.hasValue(),      # if user chose a valid option 
           Seq([
               App.localPut(Int(0), VOTING_CREDIT_SYM, credit_balance-Mul(option0_votes, option0_votes)),     # update balance
               If(Ge(credit_balance, Int(0)),             # does credit_balance this get re-evaluated? 
                    App.globalPut(option0, option0_tally.value()+option0_votes), 
                    Return(Int(0))
                 ),    
               Return(Int(1))
           ]),
           Return(Int(0))
          )
    ])
    
    
    # add up to 5 options at a time. 
    on_add_options =  Seq([
        If(Txn.application_args[1] != NULL_OPTION, 
           Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[1]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[2] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[2]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[3] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[3]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[4] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[4]), Int(0))]),
           Return(Int(1))
        ),
        If(Txn.application_args[5] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[5]), Int(0))]),
           Return(Int(1))
        ), 
        Return(Int(1))
    ])
    
    
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        App.globalPut(Bytes("asset_id"), Btoi(Txn.application_args[6])),  # should be Int 
        App.globalPut(Bytes("asset_coefficient"), Btoi(Txn.application_args[7])), 
        on_add_options,   # record the options 
        Return(Int(1))
    ])    
    
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_options"), on_add_options]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [96]:
approval_bytes, clear_state_bytes = compile()

params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

decision_name = 'letsvote'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=61, num_byte_slices=3)     # maximum sum is 64

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[4]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    # you always need to submit this many options. you can use NULL_OPTION to ingore an option 
    app_args = [decision_name.encode('utf-8'), 
                "option1".encode('utf-8'),
                "option2".encode('utf-8'),
                "option3".encode('utf-8'), 
                "NULL_OPTION".encode('utf-8'),
                "NULL_OPTION".encode('utf-8'), 
                asset_id goes here, 
                asset_coefficient_goes_here]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[4]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'FIWMYCIH43P3QHWAZSCCZT2K4IMG4LNKQUC2UH7JZV5VHOWVPJJQ'

In [130]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
print(app_id)

global_state = read_global_state(algod_client, funded_accounts[4]['pk'], app_id)
decode_keys(global_state)

14637979


[[{'key': 'option_option3', 'value': {'bytes': '', 'type': 2, 'uint': 7}},
  {'key': 'Creator',
   'value': {'bytes': 'hmXzebxJfQ9OITtFlPuRxlG9X6Sb/YeOo4wVxBw0Bh0=',
    'type': 1,
    'uint': 0}},
  {'key': 'Name', 'value': {'bytes': 'bGV0c3ZvdGU=', 'type': 1, 'uint': 0}},
  {'key': 'option_option1', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_option2', 'value': {'bytes': '', 'type': 2, 'uint': 15}}]]

In [121]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

In [99]:
# OPT-IN 

optin_tx = transaction.ApplicationOptInTxn(funded_accounts[2]['pk'], params, app_id)
optin_txid = algod_client.send_transaction(optin_tx.sign(funded_accounts[2]['sk']))

In [129]:
local_storage = algod_client.account_info(funded_accounts[2]['pk'])['apps-local-state']
local_storage

[{'id': 14637979,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 0}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [131]:
# VOTE

option_name = "option3"
votes = 1
app_args = ["vote".encode("utf-8"), option_name.encode("utf-8"), votes.to_bytes(2, "big")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[2]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[2]['sk']))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction 3MEASIX4UJHE6NF7B6LVQFQPFSB3EJUH2CV47PMYY5D5RX4XL4QA: - would result negative"}


In [1]:
### TODO 
# vote 5 options at a time 
# quadratic voting 
# ASA integration 
# verifier code 